# Shor's Algorithm

You've heard of the risk of quantum computing cracking RSA encryption - this is the reason!

## What Problem Does It Solve?
Shor’s algorithm is a **quantum algorithm** used to factor large numbers.  
Factoring large numbers is *really hard* for classical computers, which is why it's the basis of many encryption systems (like RSA).

Shor's algorithm can **efficiently find the prime factors** of a number `N`, using the power of quantum computing.

## Step 1: Finding the Period of a Function

It is the first step where Quantum computing is really beneficial. This is where we want to find something called **the period**.

Where:
- `N` is the number you want to factor
- `a` is a number less than `N` and **coprime** to it (shares no common factor other than 1)

We want to find the **period** `r` of this function — the smallest positive number such that:

a^r ≡ 1 mod N

Pick `a = 2`. Now evaluate the function:

| x  | 2^x      | f(x) = 2^x mod 15 |
|----|----------|-------------------|
| 0  | 1        | 1                 |
| 1  | 2        | 2                 |
| 2  | 4        | 4                 |
| 3  | 8        | 8                 |
| 4  | 16       | 1                 |
| 5  | 32       | 2                 |
| 6  | 64       | 4                 |
| 7  | 128      | 8                 |
| 8  | 256      | 1                 |

The output repeats every 4 steps:  

**The period `r = 4`**

**Finding the period `r` is very hard classically**, especially for large `N`.

Here is a Python example that is finding the periods using Brute force and classical computing.

In [2]:
def find_period_classically(a, N): # This is the bit you can swap for Quantum computing
    """
    Finds the period r such that a^r mod N = 1.
    This is a classical brute-force method, used for illustrating
    what the quantum part of Shor's algorithm is doing.
    
    Parameters:
        a (int): base of the function f(x) = a^x mod N
        N (int): modulus to factor (must be > a)
    
    Returns:
        int: period r such that a^r mod N == 1, or None if not found
    """
    seen = set()
    for r in range(1, N):
        value = pow(a, r, N)  # computes (a^r) mod N efficiently
        print(f"{a}^{r} mod {N} = {value}")
        if value == 1:
            return r
    return None

# Example usage:
a = 2
N = 15
period = find_period_classically(a, N)
print(f"\nPeriod r for a={a} mod N={N} is: {period}")


2^1 mod 15 = 2
2^2 mod 15 = 4
2^3 mod 15 = 8
2^4 mod 15 = 1

Period r for a=2 mod N=15 is: 4


## Step 2 Determining the Factors

This part of the problem is perfect for classical computers and does not need a quantum computer. From the period, the factors can be calculated. Below is a Python example function that can be used to calculate the factors.

In [3]:
import math

def get_factors_from_period(N, a, r):
    """
    Derives non-trivial factors of N using the period r of a^r mod N.

    Parameters:
        N (int): The number to factor
        a (int): A base such that gcd(a, N) = 1
        r (int): The period such that a^r ≡ 1 mod N

    Returns:
        (int, int): A tuple of two non-trivial factors of N, or (None, None) if not found.
    """
    # Sanity check: r must be even
    if r % 2 != 0:
        print("Period r is odd; cannot proceed.")
        return None, None

    # Compute a^(r/2) mod N
    ar2 = pow(a, r // 2, N)

    # Check if this leads to trivial factors
    if ar2 == 1 or ar2 == N - 1:
        print("a^(r/2) is ±1 mod N; trivial result. Try a different a.")
        return None, None

    # Compute the two potential factors
    factor1 = math.gcd(ar2 - 1, N)
    factor2 = math.gcd(ar2 + 1, N)

    # Check they are valid and non-trivial
    if factor1 in [1, N] or factor2 in [1, N]:
        print("GCDs resulted in trivial factors.")
        return None, None

    return factor1, factor2


## How do Quantum Computers Calculate the Period?

Quantum computers can do it efficiently using a special subroutine called:

### Quantum Phase Estimation (QPE)

Think of it like this:

> "The function `f(x)` repeats like a song loop.  
> Quantum Phase Estimation lets us ‘listen’ to all versions of the song at once and figure out the exact loop length instantly."

---

## Summary

- Shor's algorithm breaks a hard problem (factoring) into:
  - A quantum part: **Find the period `r` of f(x) = a^x mod N`**
  - A classical part: **Use `r` to calculate the factors**

- The **quantum magic** happens in finding the period using **Quantum Phase Estimation**

- Once we have the period, the rest is just number crunching

---

> That’s why quantum computers are a big deal for cryptography —  
> Shor’s algorithm could break systems like RSA **if** we can build a quantum computer big enough.


## Code Examples

The first code example illustrates how we can complete the factor finding functionality but using regular Python libraries such as math and fractions. It is similar to the code you saw earlier.

This will not scale, but illustrates mathematically what we are trying to achieve.

In [4]:
# Import required libraries for Qiskit 2.0.2
import numpy as np
from math import gcd # this is the greatest common divisor function
from fractions import Fraction

# Set a fixed seed for reproducibility
np.random.seed(42)

# Define the number to factor
N = 15
print(f"We're going to factor N = {N}")

# Step 1: Choose a random number a < N that is coprime to N
a = 7  # For demonstration, we'll choose 7 which is coprime to 15
print(f"Selected a = {a}, which is coprime to {N} (gcd = {gcd(a, N)})")

# Function to perform classical pre-processing part of Shor's algorithm
def classical_shor_part(N, a, measured_phase):
    # Convert measured phase to a fraction
    frac = Fraction(measured_phase).limit_denominator(N)
    r = frac.denominator
    print(f"Estimated period: {r}")
    
    # Check if r is even and a^(r/2) ≠ -1 (mod N)
    if r % 2 == 0 and pow(a, r//2, N) != N - 1:
        # Compute potential factors
        guessed_factor_1 = gcd(pow(a, r//2) - 1, N)
        guessed_factor_2 = gcd(pow(a, r//2) + 1, N)
        return guessed_factor_1, guessed_factor_2
    else:
        return "Failed to find factors. Try again with a different 'a' value."

# For educational purposes, demonstrate the period manually
print("\nPeriod calculation (a^x mod N):")
for x in range(10):  # Show a few values
    print(f"a^{x} mod N = {pow(a, x, N)}")

# Analyzing the pattern to find where it repeats
period = None
for r in range(1, N):
    if pow(a, r, N) == 1:
        period = r
        break
print(f"\nThe period r = {period} (this is what the quantum part finds)")

# Quantum part would compute this - we'll simulate the result
measured_phase = 0.25  # For a=7, N=15, period=4, so phase = 1/4

# Calculate the factors using the period
print("\nUsing the period to calculate factors:")
factors = classical_shor_part(N, a, measured_phase)
print(f"Factors of {N}: {factors}")


We're going to factor N = 15
Selected a = 7, which is coprime to 15 (gcd = 1)

Period calculation (a^x mod N):
a^0 mod N = 1
a^1 mod N = 7
a^2 mod N = 4
a^3 mod N = 13
a^4 mod N = 1
a^5 mod N = 7
a^6 mod N = 4
a^7 mod N = 13
a^8 mod N = 1
a^9 mod N = 7

The period r = 4 (this is what the quantum part finds)

Using the period to calculate factors:
Estimated period: 4
Factors of 15: (3, 5)


## Quantum Period-Finding Implementation

Now let's implement the quantum part of Shor's algorithm to find the period. For educational purposes, we'll demonstrate the implementation for finding the period of $a^x \bmod N$ using quantum phase estimation.

In [17]:
# Implementation of Quantum Period Finding for Shor's algorithm with Qiskit 2.0.2

def quantum_period_finding(a, N):
    """Perform quantum period finding for a^x mod N"""
    # Calculate how many qubits we need for phase estimation
    # We need enough qubits to represent numbers up to N^2
    n_count = 2 * N.bit_length()  # Number of counting qubits
    n_target = N.bit_length()    # Number of target qubits
    
    print(f"Using {n_count} counting qubits and {n_target} target qubits")
    
    # Create a quantum circuit
    qc = QuantumCircuit(n_count + n_target, n_count)
    
    # Step 1: Apply H-gates to counting qubits to create superposition
    for q in range(n_count):
        qc.h(q)
    
    # Step 2: Initial state in target register
    # Initialize target register to |1⟩
    qc.x(n_count)  # Apply X to the first target qubit
    
    # Step 3: Apply controlled-U operations
    # U|y⟩ = |a^(2^j) * y mod N⟩
    # For each counting qubit j, we need to apply a^(2^j) mod N to the target register
    for j in range(n_count):
        # Calculate a^(2^j) mod N
        power = pow(a, 2**j, N)
        
        # Ideally, we would apply a modular multiplication operation
        # This is a simplified placeholder - actual implementation would be much more complex
        # and involve modular exponentiation operations
        # In this simplified version, we'll apply a controlled phase rotation as a placeholder
        qc.cp(power * np.pi / N, j, n_count)
    
    # Step 4: Apply inverse QFT to counting qubits
    # In Qiskit 2.0.2, we need to manually implement inverse QFT
    # First, apply swaps to reverse the qubit order
    for i in range(n_count//2):
        qc.swap(i, n_count-i-1)
        
    # Then apply rotations
    for j in range(n_count):
        for k in range(j):
            qc.cp(-np.pi/float(2**(j-k)), k, j)
        qc.h(j)
    
    # Step 5: Measure counting qubits
    qc.measure(range(n_count), range(n_count))
    
    print("Circuit depth:", qc.depth())
    
    # Simulate the circuit
    simulator = AerSimulator()
    job = simulator.run(qc, shots=1024)
    result = job.result()
    counts = result.get_counts()
    
    # Process measurement results to estimate the period
    # In a real implementation, we would process the results to find the period
    # But for educational purposes, we'll just return the raw measurement results
    return counts

# Execute the quantum period finding
print("\nRunning quantum period finding (simplified):")
circuit_results = quantum_period_finding(a, N)

# Note: This is a very simplified implementation for educational purposes
# The actual implementation of Shor's algorithm's quantum part is more complex
print("\nMeasurement results from quantum circuit:")
for outcome, count in sorted(circuit_results.items(), key=lambda x: -x[1]):
    # Convert binary string to phase
    phase = int(outcome, 2) / (2**len(outcome))
    print(f"Measured: {outcome}, Count: {count}, Corresponding phase: {phase:.4f}")

print("\nIn a full Shor's implementation, we would:")
print("1. Use the most frequent phase to estimate the period")
print("2. Use continued fractions to convert the phase to a rational number")
print("3. Extract the denominator as the period")
print("4. Use the period to find factors")


Running quantum period finding (simplified):
Using 8 counting qubits and 4 target qubits
Circuit depth: 26

Measurement results from quantum circuit:
Measured: 00000000, Count: 434, Corresponding phase: 0.0000
Measured: 10000000, Count: 380, Corresponding phase: 0.5000
Measured: 01000000, Count: 144, Corresponding phase: 0.2500
Measured: 00100000, Count: 9, Corresponding phase: 0.1250
Measured: 00010000, Count: 6, Corresponding phase: 0.0625
Measured: 00110000, Count: 5, Corresponding phase: 0.1875
Measured: 11111110, Count: 5, Corresponding phase: 0.9922
Measured: 00000001, Count: 4, Corresponding phase: 0.0039
Measured: 00001000, Count: 4, Corresponding phase: 0.0312
Measured: 10000010, Count: 3, Corresponding phase: 0.5078
Measured: 00000100, Count: 3, Corresponding phase: 0.0156
Measured: 01111100, Count: 2, Corresponding phase: 0.4844
Measured: 00000010, Count: 2, Corresponding phase: 0.0078
Measured: 01000001, Count: 2, Corresponding phase: 0.2539
Measured: 11110000, Count: 2, C

## Complete Shor's Algorithm Implementation

Now let's implement a more complete version of Shor's algorithm using Qiskit 2.0.2. Note that in practice, actual factorization of numbers beyond very small examples would require significant quantum resources that aren't available today.

In [15]:
# A more complete implementation of Shor's algorithm for Qiskit 2.0.2
# Note: Still simplified compared to a production implementation

from math import gcd
import numpy as np
from fractions import Fraction
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT

def qpe_amod15(a):
    """Creates a phase estimation circuit for finding the period of a^x mod 15"""
    n_count = 8  # Number of counting qubits
    
    # Create a quantum circuit with enough qubits for QFT plus 4 target qubits
    qc = QuantumCircuit(n_count + 4, n_count)
    
    # Initialize target qubits to |1>
    qc.x(n_count)
    
    # Apply Hadamards to counting qubits
    for q in range(n_count):
        qc.h(q)
    
    # Apply controlled powers of U
    # U|y> = |a*y mod 15>
    # Instead of implementing the full modular multiplication,
    # we'll use some circuit identities specific to our example
    
    # Hard-coding implementations for specific values of 'a'
    # For a=2, a=4, a=7, a=8, a=11, a=13 (common coprime values to 15)
    if a == 2:  # Use a=2 as an example
        # Apply controlled-U operations
        for i in range(n_count):
            # For a=2, we need to apply a controlled-squaring mod 15 operation 2^i times
            for _ in range(2**i):
                # We'll use a simplified squaring operation
                # This is a placeholder and not a true modular multiplication
                qc.cswap(i, n_count+1, n_count+3)  # Controlled-SWAP
                qc.cswap(i, n_count+0, n_count+2)  # Controlled-SWAP
    elif a == 7:  # For a=7, we'd have a different circuit
        # This is simplified - for a=7 we'd need different quantum operations
        for i in range(n_count):
            # Placeholder operations for a=7 mod 15
            qc.cp(np.pi/2, i, n_count)
            qc.cp(np.pi/4, i, n_count+1)
    else:
        # For other values, we'd need appropriate circuits
        # This is a simplified placeholder
        for i in range(n_count):
            qc.cp(a * np.pi / 8, i, n_count)
    
    # Apply inverse QFT to counting qubits
    # Manual implementation of inverse QFT
    # First, apply swaps to reverse the qubit order
    for i in range(n_count//2):
        qc.swap(i, n_count-i-1)
        
    # Then apply rotations
    for j in range(n_count):
        for k in range(j):
            qc.cp(-np.pi/float(2**(j-k)), k, j)
        qc.h(j)
    
    # Measure counting qubits
    qc.measure(range(n_count), range(n_count))
    
    return qc

def shor_algorithm(N):
    """Implements Shor's algorithm to factor N"""
    
    # Step 1: Check if N is even
    if N % 2 == 0:
        return 2, N//2
    
    # Step 2: Check if N is a prime power
    # (simplified for this example)
    
    # Step 3: Choose a random number 1 < a < N
    np.random.seed(42)  # For reproducibility
    while True:
        a = np.random.randint(2, N)
        if gcd(a, N) != 1:
            # If gcd(a,N) > 1, we found a factor already
            return gcd(a, N), N // gcd(a, N)
        
        print(f"Selected a = {a}")
        
        # Step 4: Use quantum period finding
        # Create the circuit for period finding
        qc = qpe_amod15(a)
        
        # Simulate the circuit
        print("Executing quantum circuit...")
        simulator = AerSimulator()
        job = simulator.run(qc, shots=1000)
        result = job.result()
        counts = result.get_counts()
        
        # Step 5: Process the results to find the period
        # Get the phase with highest frequency
        phase = int(max(counts.items(), key=lambda x: x[1])[0], 2) / 2**qc.num_clbits
        print(f"Measured phase: {phase}")
        
        # Convert phase to fraction to estimate period
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator
        print(f"Estimated period: {r}")
        
        # Step 6: Check if period is useful for finding factors
        if r % 2 == 0 and pow(a, r//2, N) != N-1:
            # Compute potential factors
            p = gcd(pow(a, r//2) - 1, N)
            q = gcd(pow(a, r//2) + 1, N)
            if p != 1 and p != N and p * (N//p) == N:
                return p, N//p
            if q != 1 and q != N and q * (N//q) == N:
                return q, N//q
        
        print("This attempt did not yield factors. Trying again...")
        
        # For educational purposes, limit attempts
        break
    
    return "Failed to find factors"

# Run Shor's algorithm to factor N=15
N = 15
print(f"\nRunning complete Shor's algorithm to factor {N}...")
factors = shor_algorithm(N)
print(f"Factors of {N}: {factors}")


Running complete Shor's algorithm to factor 15...
Selected a = 8
Executing quantum circuit...
Measured phase: 0.66796875
Estimated period: 3
This attempt did not yield factors. Trying again...
Factors of 15: Failed to find factors


In [16]:
# Visualize a simplified version of the quantum circuit used in Shor's algorithm

# Define a simpler visualization circuit
visualization_qc = QuantumCircuit(6, 4)  # 4 counting qubits and 2 target qubits
visualization_qc.x(4)  # Initialize target qubit to |1>

# Apply Hadamards to counting qubits
for q in range(4):
    visualization_qc.h(q)

# Apply some controlled operations (simplified for visualization)
for i in range(4):
    visualization_qc.cp(np.pi/(2**(3-i)), i, 4)

# Apply inverse QFT manually
# First, apply swaps to reverse the qubit order
n_count = 4  # Number of counting qubits
for i in range(n_count//2):
    visualization_qc.swap(i, n_count-i-1)
    
# Then apply rotations
for j in range(n_count):
    for k in range(j):
        visualization_qc.cp(-np.pi/float(2**(j-k)), k, j)
    visualization_qc.h(j)

# Measure counting qubits
visualization_qc.measure(range(4), range(4))

# Draw the circuit
print("Simplified visualization of Shor's algorithm quantum circuit:")
try:
    # Try to render with text output first (more compatible)
    circuit_diagram = visualization_qc.draw(output='text', fold=-1)
    print(circuit_diagram)
    
    # Optionally try to render with matplotlib
    visualization_qc.draw(output='mpl', fold=-1)
except Exception as e:
    print(f"Error rendering circuit: {e}")
    print("Circuit information:")
    print(f"Number of qubits: {visualization_qc.num_qubits}")
    print(f"Number of classical bits: {visualization_qc.num_clbits}")
    print(f"Circuit depth: {visualization_qc.depth()}")

Simplified visualization of Shor's algorithm quantum circuit:
     ┌───┐                                     ┌───┐                                                            ┌─┐                        
q_0: ┤ H ├─■─────────────────────────────────X─┤ H ├─■──────────────■───────────────────■───────────────────────┤M├────────────────────────
     ├───┤ │                                 │ └───┘ │P(-π/2) ┌───┐ │                   │                       └╥┘          ┌─┐           
q_1: ┤ H ├─┼────────■───────────────────X────┼───────■────────┤ H ├─┼─────────■─────────┼──────────────■─────────╫───────────┤M├───────────
     ├───┤ │        │                   │    │                └───┘ │P(-π/4)  │P(-π/2)  │        ┌───┐ │         ║           └╥┘     ┌─┐   
q_2: ┤ H ├─┼────────┼────────■──────────X────┼──────────────────────■─────────■─────────┼────────┤ H ├─┼─────────╫──■─────────╫──────┤M├───
     ├───┤ │        │        │               │                                          │P(-π/8) └